In [1]:
import random
import json
import numpy as np
import pickle

import nltk
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append(((word_list), intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word)
         for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# training = []
# output_empty = [0] * len(classes)

# for document in documents:
#     bag = []
#     word_patterns = document[0]
#     word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
#     for word in words:
#         bag.append(1) if word in word_patterns else bag.append(0)

#     output_row = list(output_empty)
#     output_row[classes.index(document[1])] = 1
#     training.append([bag, output_row])

# random.shuffle(training)
# training = np.array(training)

# train_x = list(training[:, 0])
# train_y = list(training[:, 1])

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(
        word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)

# Convert training data to NumPy arrays
train_x = np.array([sample[0] for sample in training])
train_y = np.array([sample[1] for sample in training])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)
# model.save('chatbotmodel.h5', hist)
model.save('chatbotmodel.keras')
print('Done')




Epoch 1/200


31/31 [==============================] - 1s 2ms/step - loss: 3.5291 - accuracy: 0.0464
Epoch 2/200
31/31 [==============================] - 0s 2ms/step - loss: 3.5137 - accuracy: 0.0464
Epoch 3/200
31/31 [==============================] - 0s 2ms/step - loss: 3.4212 - accuracy: 0.1126
Epoch 4/200
31/31 [==============================] - 0s 2ms/step - loss: 3.3408 - accuracy: 0.1258
Epoch 5/200
31/31 [==============================] - 0s 2ms/step - loss: 3.2708 - accuracy: 0.1722
Epoch 6/200
31/31 [==============================] - 0s 2ms/step - loss: 3.1695 - accuracy: 0.1854
Epoch 7/200
31/31 [==============================] - 0s 2ms/step - loss: 3.0329 - accuracy: 0.1656
Epoch 8/200
31/31 [==============================] - 0s 2ms/step - loss: 2.9086 - accuracy: 0.1722
Epoch 9/200
31/31 [==============================] - 0s 2ms/step - loss: 2.7617 - accuracy: 0.2450
Epoch 10/200
31/31 [==============================] - 0s 2ms/step - loss: 2.6471 - accuracy: 0.2517
Epoch